In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import json


In [ ]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# OpenAI 클라이언트 생성 (API 키 설정)
client = OpenAI(api_key=api_key)

# CSV 파일 불러오기
df = pd.read_csv("hanja.csv")  # 파일 경로를 여기에 입력

In [ ]:
df.head()

In [ ]:
# 'hanja'와 'radical' 컬럼만 추출
hanjas = df['hanja'].dropna().unique()
radicals = df['radical'].dropna().unique()

# 중복 제거된 한자 및 부수 리스트
unique_items = list(set(hanjas.tolist() + radicals.tolist()))

In [ ]:
print(unique_items[:10])
print(len(unique_items))

In [ ]:
# 임베딩 결과 저장 딕셔너리
embeddings = {}

In [ ]:
# ✅ 배치 기반 임베딩 함수
def embed_texts_batch(text_list, embeddings={}, batch_size=100):
    # 중복 요청 방지: 이미 있는 건 건너뜀
    new_texts = [text for text in text_list if text not in embeddings]

    for i in tqdm(range(0, len(new_texts), batch_size), desc="임베딩 중"):
        batch = new_texts[i:i+batch_size]
        try:
            response = client.embeddings.create(
                model="text-embedding-3-small",
                input=batch
            )
            for text, result in zip(batch, response.data):
                embeddings[text] = result.embedding
        except Exception as e:
            print(f"배치 {i // batch_size + 1} 임베딩 실패: {e}")
    
    return embeddings

# 🧠 임베딩 수행
embeddings = embed_texts_batch(unique_items, embeddings)

In [ ]:
# 결과 저장 (optional)
with open("hanja_radical_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(embeddings, f, ensure_ascii=False, indent=2)

print("✅ 임베딩 완료 및 저장됨.")

In [ ]:
type(embeddings)

In [ ]:
len(embeddings)


In [ ]:

import matplotlib.pyplot as plt
from matplotlib import rc  ### mac 환경에서 한글 폰트 설정
rc('font', family='AppleGothic') 			## 이 두 줄을 
plt.rcParams['axes.unicode_minus'] = False  ## 추가해줍니다. 
